Process output from Arpeggio<br>
Code from Bashour et al<br>
13/2/25

In [1]:
import os
import json
from operator import itemgetter
from collections import defaultdict, Counter
import numpy as np
#from multiprocessing import Pool
import pandas as pd

In [89]:
dataset_name = 'vhh_tsd'
path = '/vols/bitbucket/gordon/tnp_data/arpeggio/'+dataset_name+'/'
files = [path + f for f in os.listdir(path) if f.endswith('.json')]

In [90]:
len(files)

36

In [91]:
def flatten(t):
    return [item for sublist in t for item in sublist]


def run(f):
    
    #print(f)
    d = {}
    with open(f) as rf:
        json_d = json.load(rf)

        contacts = Counter(flatten([j['contact'] for j in json_d]))

        interactions = [itemgetter('distance', 'interacting_entities', 'type')(j) for j in json_d]
        interacting_entities = Counter([i[1] for i in interactions])
        types = Counter([i[2] for i in interactions])

        d.update(contacts)
        d.update(interacting_entities)
        d.update(types)
        d.update({'distance': np.mean([i[0] for i in interactions])})
        
    return d

In [92]:
# pool = Pool(processes = cpus)
# results = pool.map(run, files)
# pool.close()
# pool.join()

results = []
for file in files:
    output = run(file)
    results.append(output)

print(len(results))

36


In [93]:
d_protint = defaultdict(dict)

for n, (f, r) in enumerate(zip(files, results)):
    #print(n)
    name = f.split('/')[-1].split('.')[0]
    d_protint[name] = r

In [94]:
df = pd.DataFrame(d_protint.values(), index=d_protint.keys())

In [95]:
df = df.reset_index()
df.columns

Index(['index', 'vdw', 'hydrophobic', 'proximal', 'weak_polar', 'polar',
       'vdw_clash', 'hbond', 'carbonyl', 'weak_hbond', 'aromatic', 'ionic',
       'covalent', 'EE', 'CARBONPI', 'DONORPI', 'METSULPHURPI', 'CATIONPI',
       'AMIDEAMIDE', 'AMIDERING', 'INTRA_SELECTION', 'atom-atom',
       'plane-plane', 'atom-plane', 'group-group', 'group-plane', 'distance',
       'ET', 'FT', 'FE', 'EF', 'OF', 'OE', 'OT'],
      dtype='object')

In [96]:
df = df.rename(columns={'index':'SeqID'})

In [97]:
df

,SeqID,vdw,hydrophobic,proximal,weak_polar,polar,vdw_clash,hbond,carbonyl,weak_hbond,...,group-group,group-plane,distance,ET,FT,FE,EF,OF,OE,OT
0,seq27,74,281,4035,110,155,150,122,26,78,...,8,4,4.305504,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,seq26,61,285,4114,92,157,142,127,17,77,...,8,3,4.306677,2.0,2.0,2.0,2.0,NaN,NaN,NaN
2,seq4,76,339,4597,117,167,177,129,24,96,...,8,2,4.312330,NaN,NaN,NaN,1.0,6.0,1.0,NaN
3,seq7,68,318,4354,110,162,155,130,26,76,...,10,1,4.308247,2.0,2.0,2.0,3.0,NaN,3.0,NaN
4,seq36,67,316,4308,101,156,154,125,19,83,...,5,3,4.313879,3.0,2.0,2.0,3.0,NaN,1.0,1.0
5,seq30,71,279,4038,106,162,155,127,23,79,...,12,6,4.299134,NaN,NaN,NaN,NaN,NaN,1.0,NaN
6,seq33,75,290,4314,103,172,179,138,23,86,...,14,1,4.301606,2.0,2.0,1.0,1.0,NaN,1.0,2.0
7,seq20,64,331,4599,113,166,187,142,19,84,...,9,1,4.300970,2.0,3.0,2.0,3.0,NaN,1.0,1.0
8,seq1,71,293,3917,104,145,158,119,24,77,...,4,5,4.298707,2.0,2.0,1.0,1.0,NaN,NaN,NaN
9,seq29,63,266,3922,94,150,136,127,14,70,...,9,3,4.309165,NaN,NaN,NaN,1.0,NaN,1.0,NaN


In [98]:
df.to_csv('/data/localhost/gordon/TNP_Project/RESULTS/greiff_results/arpeggio/'+dataset_name+'_arpeggio.csv',index=False)